# Entity-Level Evaluation

In [94]:
# The script is downloaded from GSAP-NER: https://github.com/ottowg/gsap-ner/blob/emnlp_submission/evaluation/evaluation_methods.py
from bisect import bisect_left, bisect_right
import pandas as pd
from IPython.display import display,HTML

def calc_scores(gold, prediction):
    scores = prec_recall(gold, prediction, "precision")\
                .join(prec_recall(gold, prediction, "recall"))
    add_f1(scores)
    return scores

def prec_recall(gold, prediction, score_name):
    if score_name == "recall":
        prediction, gold = gold, prediction
    elif score_name != "precision":
        raise Exception("no valid score")
    scores_interim = remap_annos_best_match_many_documents(gold,
                                                prediction)
    scores_interim = pd.DataFrame(scores_interim)
    label_p = scores_interim.label.value_counts()
    p = label_p.sum()
    scores_interim["partial_tp"] = scores_interim.label == scores_interim.label_mapped
    scores_interim["exact_tp"] = (scores_interim.label == scores_interim.label_mapped)\
                                 & (scores_interim.match_type == "exact_match")
    partial_score = (scores_interim.groupby("label").partial_tp.sum() /\
                     label_p).rename(f"partial_{score_name}")
    partial_score.loc["all"] = scores_interim.partial_tp.sum() / p
    exact_score = (scores_interim.groupby("label").exact_tp.sum() / label_p)\
                        .rename(f"exact_{score_name}")
    exact_score.loc["all"] = scores_interim.exact_tp.sum() / p
    score = partial_score.to_frame().join(exact_score)
    return score


def remap_annos_best_match_df(doc_annos, exact=False):
    gold = doc_annos.gold
    pred = doc_annos.pred
    result = remap_annos_best_match_many_documents(gold, pred, exact=False)
    return result


def remap_annos_best_match_many_documents(gold, pred, exact=False):
    """
    each annotation (gold and prediction) need to contain: "begin", "end"
    The documents will be grouped by "id" before matching.
    Each "id" represents one document.
    """
    document_annos = get_document_dict(gold, pred)
    all_annos = []
    for doc, annos in document_annos.items():
        #print(doc)
        #print(len(annos["gold"]))
        #print(len(annos["prediction"]))
        remaped = remap_annos_best_match(annos["gold"], annos["prediction"], exact=exact)
        all_annos.extend(remaped)
    return all_annos
    
def get_document_dict(gold, pred):
    idents = {g["doc_id"] for g in gold}
    idents |= {p["doc_id"] for p in pred}
    print("n_documents:", len(idents))
    by_ident = {ident: dict(gold=[], prediction=[])
                for ident in idents}
    for g in gold:
        by_ident[g["doc_id"]]["gold"].append(g)
    for p in pred:
        by_ident[p["doc_id"]]["prediction"].append(p)
    return by_ident
    
    
def remap_annos_best_match(gold, pred, exact=False):
    pred_remapped = []
    for idx, g in enumerate(gold):
        g["index"] = idx
    gold_begin = sorted(gold, key=lambda x:x["begin"])
    gold_end = sorted(gold, key=lambda x:x["end"])
    for p in pred:
        overlaps = get_overlapping(p, gold_begin, gold_end)
        overlaps = [gold[idx] for idx in overlaps]
        # find perfect match
        match_type = "no_match"
        matched = []
        # find perfect match
        if not matched:
            overlaps_selected = [o for o in overlaps if o["label"] == p["label"]
                                 and (o["begin"] == p["begin"] and o["end"] == p["end"])
                                ]
            if overlaps_selected:
                matched = overlaps_selected
                match_type = "exact_match"
        if not matched and not exact:
            # find same label but only overlapping span
            overlaps_selected = [o for o in overlaps if o["label"] == p["label"]
                        and (o["begin"] != p["begin"] or o["end"] != p["end"])
                       ]
            if overlaps_selected:
                matched = overlaps_selected
                match_type = "partly_match"
        if not matched:
            # find different label but same span
            overlaps_selected = [o for o in overlaps if o["label"] != p["label"]
                        and (o["begin"] == p["begin"] and o["end"] == p["end"])
                       ]
            if overlaps_selected:
                matched = overlaps_selected
                match_type = "exact_span_match"
        if not matched and not exact:
            # find different label and different overlapping span
            overlaps_selected = [o for o in overlaps if o["label"] != p["label"]
                        and (o["begin"] != p["begin"] or o["end"] != p["end"])
                       ]
            if overlaps_selected:
                matched = overlaps_selected
                match_type = "partly_span_match"
        match = {} if not matched else matched[0]
        match = {k:v for k, v in match.items()}
        labels = sorted(list(set([a["label"] for a in matched])))
        labels = " ".join(labels)
        p = {k:v for k, v in p.items()}
        p["text_mapped"] = match.get("text")
        p["begin_mapped"] = match.get("begin")
        p["end_mapped"] = match.get("end")
        p["label_mapped"] = match.get("label", "not_found")
        p["labels_mapped"] = labels if labels else "not_found"
        p["n_matched"] = len(matched)
        p["match_type"] = match_type
        pred_remapped.append(p)
    return pred_remapped

def get_overlapping(anno, annos_begin, annos_end):
    # all annos with smaller end as begining of query anno
    anno_idx_end_max = bisect_right(annos_end, anno["begin"], key=lambda x:x["end"])
    # all annos with smaller beginning as ending of query anno
    anno_idx_begin_min = bisect_left(annos_begin, anno["end"], key=lambda x:x["begin"])
    #anno_idx_begin_min, anno_idx_end_max

    matches = {a["index"] for a in annos_end[anno_idx_end_max:]} &\
              {a["index"] for a in annos_begin[:anno_idx_begin_min]}
    return matches

def add_f1(scores):
    for t in ["partial", "exact"]:
        scores[f"{t}_f1"] = 2 * (scores[f"{t}_precision"] * scores[f"{t}_recall"]) /\
                                (scores[f"{t}_precision"] + scores[f"{t}_recall"])


def print_annos(annos):
    for a in annos:
        print("\t", end="")
        print_anno(a)

def print_anno(anno):
    print(f'{anno["begin"]} {anno["end"]} {anno["label"]}')

# Consider our Dataset

In [95]:
import os
from webanno_tsv import webanno_tsv_read_file

def convert_to_gold_annos(tsv_path):
    document = webanno_tsv_read_file(tsv_path)
    gold_annos = []
    doc_id = os.path.splitext(os.path.basename(tsv_path))[0]

    for anno in document.annotations:
        tokens = anno.tokens
        begin = tokens[0].start
        end = tokens[-1].end
        text = ' '.join([token.text for token in tokens])
        gold_anno = {
            "doc_id": doc_id,
            "begin": begin,
            "end": end,
            "label": anno.label,
            "text": text
        }
        gold_annos.append(gold_anno)
    
    return gold_annos

## deepseek-chat

In [96]:
# Directory paths
ref_dir = '../data/test_labeled'
src_dir_deepseek = '../results/deepseek-chat/test_unlabeled'

# Process all TSV files in the reference directory
all_gold_annos = []
all_pred_annos = []

# Get all TSV files in the reference directory
ref_files = [f for f in os.listdir(ref_dir) if f.endswith('.tsv')]

for ref_file in ref_files:
    ref_path = os.path.join(ref_dir, ref_file)
    src_path_deepseek = os.path.join(src_dir_deepseek, ref_file)
    
    # Process reference annotations
    if os.path.exists(ref_path):
        all_gold_annos.extend(convert_to_gold_annos(ref_path))
    
    # Process prediction annotations
    if os.path.exists(src_path_deepseek):
        all_pred_annos.extend(convert_to_gold_annos(src_path_deepseek))
    
scores = calc_scores(all_gold_annos, all_pred_annos)
print("\n=== Evaluation Result ===")
display(scores)

n_documents: 40
n_documents: 40

=== Evaluation Result ===


,partial_precision,exact_precision,partial_recall,exact_recall,partial_f1,exact_f1
label,,,,,,
CONFERENCE,0.500000,0.250000,0.015385,0.005128,0.029851,0.010050
DATASET,0.200000,0.000000,0.007605,0.000000,0.014652,NaN
EVALMETRIC,0.000000,0.000000,0.000000,0.000000,NaN,NaN
LICENSE,0.000000,0.000000,0.000000,0.000000,NaN,NaN
PROGLANG,0.000000,0.000000,0.000000,0.000000,NaN,NaN
PROJECT,0.000000,0.000000,0.000000,0.000000,NaN,NaN
PUBLICATION,0.709677,0.193548,0.060773,0.016575,0.111959,0.030534
SOFTWARE,0.200000,0.125000,0.009709,0.006068,0.018519,0.011574
WORKSHOP,1.000000,0.000000,0.027027,0.000000,0.052632,NaN


In [97]:
# # Convert to DataFrames
# gold_df = pd.DataFrame(all_gold_annos)
# pred_df = pd.DataFrame(all_pred_annos)

# # Display the DataFrames
# print("Gold Annotations:")
# print(gold_df.head())
# print("\nShape:", gold_df.shape)

# print("\nPredicted Annotations:")
# print(pred_df.head())
# print("\nShape:", pred_df.shape)

## Meta-Llama-3-8B-Instruct

In [98]:
# Directory paths
ref_dir = '../data/test_labeled'
src_dir_deepseek = '../results/Meta-Llama-3-8B-Instruct/test_unlabeled'

# Process all TSV files in the reference directory
all_gold_annos = []
all_pred_annos = []

# Get all TSV files in the reference directory
ref_files = [f for f in os.listdir(ref_dir) if f.endswith('.tsv')]

for ref_file in ref_files:
    ref_path = os.path.join(ref_dir, ref_file)
    src_path_deepseek = os.path.join(src_dir_deepseek, ref_file)
    
    # Process reference annotations
    if os.path.exists(ref_path):
        all_gold_annos.extend(convert_to_gold_annos(ref_path))
    
    # Process prediction annotations
    if os.path.exists(src_path_deepseek):
        all_pred_annos.extend(convert_to_gold_annos(src_path_deepseek))
    
scores = calc_scores(all_gold_annos, all_pred_annos)
print("\n=== Evaluation Result ===")
display(scores)

n_documents: 40
n_documents: 40

=== Evaluation Result ===


,partial_precision,exact_precision,partial_recall,exact_recall,partial_f1,exact_f1
label,,,,,,
DATASET,0.000000,0.0,0.000000,0.0,NaN,NaN
PUBLICATION,0.023256,0.0,0.002762,0.0,0.004938,NaN
all,0.013514,0.0,0.000501,0.0,0.000967,NaN


## Mistral-7B-Instruct-v0.3

In [99]:
# Directory paths
ref_dir = '../data/test_labeled'
src_dir_deepseek = '../results/Mistral-7B-Instruct-v0.3/test_unlabeled'

# Process all TSV files in the reference directory
all_gold_annos = []
all_pred_annos = []

# Get all TSV files in the reference directory
ref_files = [f for f in os.listdir(ref_dir) if f.endswith('.tsv')]

for ref_file in ref_files:
    ref_path = os.path.join(ref_dir, ref_file)
    src_path_deepseek = os.path.join(src_dir_deepseek, ref_file)
    
    # Process reference annotations
    if os.path.exists(ref_path):
        all_gold_annos.extend(convert_to_gold_annos(ref_path))
    
    # Process prediction annotations
    if os.path.exists(src_path_deepseek):
        all_pred_annos.extend(convert_to_gold_annos(src_path_deepseek))
    
scores = calc_scores(all_gold_annos, all_pred_annos)
print("\n=== Evaluation Result ===")
display(scores)

n_documents: 40
n_documents: 40

=== Evaluation Result ===


,partial_precision,exact_precision,partial_recall,exact_recall,partial_f1,exact_f1
label,,,,,,
CONFERENCE,0.000000,0.000000,0.000000,0.000000,NaN,NaN
DATASET,0.500000,0.000000,0.011407,0.000000,0.022305,NaN
EVALMETRIC,0.000000,0.000000,0.000000,0.000000,NaN,NaN
LICENSE,0.250000,0.000000,0.038462,0.000000,0.066667,NaN
ONTOLOGY,0.000000,0.000000,0.000000,0.000000,NaN,NaN
PROGLANG,0.000000,0.000000,0.000000,0.000000,NaN,NaN
PROJECT,0.000000,0.000000,0.000000,0.000000,NaN,NaN
PUBLICATION,0.391304,0.043478,0.024862,0.002762,0.046753,0.005195
SOFTWARE,0.000000,0.000000,0.000000,0.000000,NaN,NaN
